In [ ]:
##################################################
### Author: Anthony Igel                       ###
### Team: Category Management Transformation   ###
### Project: Developing practical Python Tools ###
### Purpose: Data Exploration                  ###
### Date: 05/23/2018                           ###
##################################################

### Data exploration in Python
#################################################################################################################################

######################################################################
########                     Import Modules                   ########
######################################################################
import py_effo as py_effo
from py_effo.oracle_connection import OracleConnection

### pandas
# Pandas is for structured data operations and manipulations, extensively used for data preparation
import pandas as pd

### use_bottleneck & use_numexpr both accelerate computation in panda functions
pd.set_option('compute.use_bottleneck', False)
pd.set_option('compute.use_numexpr', False)

### numpy
# NumPy stands for Numerical Python, a library contains basic linear algebra functions, Fourier Transforms and advanced random
# number capabilities
import numpy as np 

### Scipy
# Scipy performs a host of statistical calculations, built on top of Numpy, thus we do not need to import Numpy as all Numpy
# functions are contained in Scipy
# https://oneau.wordpress.com/2011/02/28/simple-statistics-with-scipy/
import scipy as sp

### Statsmodels
import statsmodels.formula.api as sm

### Matplotlib
# Matplotlib is a Python based plotting library with complete 2D support and limited 3D support
%matplotlib inline
import matplotlib as mlb
import matplotlib.pyplot as plt

### Seaborn
# Seaborn is a Python visualization library based on Matplolib, providing high-level interface for statistcial graphing
# Seaborn supports numpy and pandas data structures as well as statistical routines from scipy and statsmodels
# Note: https://seaborn.pydata.org/introduction.html
import seaborn as sns

### String
# Allows for more flexible solutions for dealing with string characters
import string as st

In [ ]:
######################################################################
########                     Import Data                      ########
######################################################################

### Commodity by Division, Fiscal Year statistics
ora_con = OracleConnection(user = 'an_rt_ws106')
comm_fy_div_metrics = ora_con.import_table('CM_TIERS_COMM_FY_DIV_METRICS')

### Mega-Category to Commodity Mapping file
mega_category_mappings = pd.read_csv("/nfs/analysis/analysis/kroger/category_management_transformation/mini_hack_days/python/category_survey_product_mapping.csv")

In [ ]:
######################################################################
########                  Data Expoloration                   ########
######################################################################
# http://python-reference.readthedocs.io/en/latest/index.html
# https://github.com/lis365b/data_analysis/blob/master/data_prep_python_cheatsheet.md
# https://www.analyticsvidhya.com/wp-content/uploads/2015/06/infographics-final.jpg
# http://www.statsmodels.org/stable/index.html

######## head() ########
### Similar to r, you can use the head() function to view the top values for the data set specifed
### You can specifiy how many records you want to view by using an integer value as the argument
### Additonally, you can use the function tail() to view the end of the data frame

# Example 1 - head() and tail() functions
print('Example 1')
print("comm_fy_div_metrics")
print(comm_fy_div_metrics.head(5))
print()
print("mega_category_mappings")
print(mega_category_mappings.tail(5))
print()

# Example 2 - Using row identifiers
print('Example 2')
print(comm_fy_div_metrics[:2])

In [ ]:
######## len() ########
### The length function in Python acts just as it does in r, counting each row

# Example
print("comm_fy_div_metrics length")
print(len(comm_fy_div_metrics))
print()
print("mega_category_mappings length")
print(len(mega_category_mappings))

In [ ]:
######## info() ########
### The info function allows the user to view information on the data frame's data types

# Example
comm_fy_div_metrics.info()
print()
mega_category_mappings.info()

In [ ]:
######## Data Frame Information ########
### You can use the keys(), values(), and items() functions to return informtation about columns and rows
### items() and values() are used for lists, not data frames

# Example 1 - Return column names
for k in comm_fy_div_metrics.keys():
    print(k)

print()
### You can also utilize functions from the pandas module to extract metadata information
print('Extract data within the data structure')
print(comm_fy_div_metrics.values)

In [ ]:
######## isnull ########
### The isnull function is a boolean evaluator to determine if the record is null or contains a value (str, int, etc.)

# Example 1 - All records with boolean evalulation of isnull()
print(comm_fy_div_metrics.isnull().head(5))

# Example 2 - Summary count of each variable with sum of values with isnull() = True
print(comm_fy_div_metrics.isnull().sum())

print()
print(mega_category_mappings.isnull().sum())

In [ ]:
######## Replacing Missing Values with Zeroes ########
### You can create a simple for() loop to replace missing numeric values with zeroes

# Example 
for i in range(0): # Only calling the numeric variable
    mega_category_mappings.iloc[:, [i]].fillna(value = 0, inplace = True)

### Then verify that it worked properly
print(mega_category_mappings.isnull().sum())

In [ ]:
######## Modifying Data Types ########
### You can create a simple for() loop to modify data types

# Example 1
for i in range(1, 5): # Only calling the object variables
    mega_category_mappings.iloc[:, [i]].apply(np.str)
    
# Example 2
mega_category_mappings["prod_merch_l20_code"] = mega_category_mappings['prod_merch_l20_code'].astype('str')

# Example 3
comm_fy_div_metrics["UNITS"] = comm_fy_div_metrics['UNITS'].astype('float')
comm_fy_div_metrics["VISITS"] = comm_fy_div_metrics['VISITS'].astype('float')
comm_fy_div_metrics["NUM_PRICEZONE"] = comm_fy_div_metrics['NUM_PRICEZONE'].astype('float')

In [ ]:
print(mega_category_mappings.dtypes)
print()
print(comm_fy_div_metrics.dtypes)

In [ ]:
######## Duplicate Detection ########
### To find duplicate values you can do it by two methods

# Example 1 - Count unique values of one column
print('Example 1')
print(mega_category_mappings.groupby('mega_category_map').nunique().head(5))
print()

# Example 2 - Count unique values of one column and return summary information
print('Example 2')
print(mega_category_mappings.groupby('mega_category_map').nunique().sum())
print()

# Example 3 - Added layer of dimensionality to display unique values
print('Example 3')
print(mega_category_mappings.groupby('mega_category_map').prod_merch_l20_code.nunique().sum())
print()

# Example 4 - Show duplicate values for one column
print('Example 4')
print((i for _, in mega_category_mappings.groupby('mega_category_map') if len(i) > 1))


In [ ]:
######################################################################
########                  Data Manipulation                   ########
######################################################################

### In mega_category_mappings it appears that our Commodity Code is numerical, as opposed to a zero-padded string
### We might have whitespace in the column names so we should change that
### Let's fix those

# http://pbpython.com/pandas_dtypes.html
# https://docs.python.org/3.6/library/stdtypes.html#
# https://www.programiz.com/python-programming/methods/string/format
# https://docs.python.org/3/library/string.html#string-formatting

### This will ensure that all column names are stripped of whitespace
mega_category_mappings.rename(columns = lambda x: x.strip(), inplace = True)

### We can use the function pad), with our width of left-pad is our argument
# Example 
mega_category_mappings["prod_merch_l20_code"] = mega_category_mappings['prod_merch_l20_code'].str.pad(3, side = 'left', fillchar = '0')    


### We can also adjust the case of our metrics table columns
comm_fy_div_metrics.rename(columns = lambda x: x.lower(), inplace = True)


In [ ]:
print(mega_category_mappings.head(5))
print()
print(comm_fy_div_metrics.head(5))

In [ ]:
######## Data Descriptions ########
### It is easy to call quick statistics of data frames using the describe() function
print('Commodity by Division FY')
print(comm_fy_div_metrics.describe())
print()
print('Commodity by Division FY w/ Percentiles')
print(comm_fy_div_metrics.describe(percentiles=[.05, .25, .75, .95]))


### If your data frame contains numerical and categorical values, Python will natively only display numerical values
### Use the include = [''] argument to alter this decision (include = ['all'], include = ['object'], include = ['number'])

In [ ]:
######################################################################
########                Joining data frames                   ########
######################################################################

### From the module pandas, we can utilize their merge() function to join our metrics and mapping data frames
full_data = pd.merge(comm_fy_div_metrics, mega_category_mappings, how = 'left', on = 'prod_merch_l20_code')

del full_data['prod_merch_l20_desc'] # You can specify particular columns to delete from data frames using "del"
del full_data['mega_category_map_nf']
### Let's check to see if it was done correctly
print('Length of Metrics Data')
print(len(comm_fy_div_metrics))
print()
print('Length of Mapping Data')
print(len(mega_category_mappings))
print()
print('Length of Full Data')
print(len(full_data))

In [ ]:
### Since not every prod_merch_l20_code has a mapping for their Category Role or Mega-Category, there are many 'Nan' values
print('Summary of Columns with Null Values')
print(full_data.isnull().sum())
print()
full_data['mega_category_map'] = full_data.mega_category_map.apply(lambda x: x if not pd.isnull(x) else 'NA')
full_data['role_name'] = full_data.role_name.apply(lambda x: x if not pd.isnull(x) else 'NA')
print('After Cleaning Summary')
print(full_data.isnull().sum())

In [ ]:
######################################################################
########                   Viewing the Data                   ########
######################################################################

######## sort_values() ########

# Example 1
### Sort by Spend
print("Sort by Spend")
print(full_data.sort_values(by = 'spend', ascending = False).head(3))

print()
### Sort by Number of Price Zones
print("Sort by Number of Price Zones")
print(full_data.sort_values(by = "num_pricezone", ascending = False).head(3))

In [ ]:
######################################################################
########                Graphing Distributions                ########
######################################################################

### Import py_effo's branding 
import py_effo.branding as branding

# View the colors as hex_values
colors = branding.Colors()
colors

#for key in colors.colors:
#    value = colors.colors[key]
#    print(key + ' -> ' + str(value))

In [ ]:
### We can view what our current palette is
current_palette = sns.color_palette()
sns.palplot(current_palette)

#sns.color_palette((244, 51, 171), (0, 189, 179), (255, 215, 69), (2, 104, 209), (169, 19, 187))

In [ ]:
######################################################################
########                 Plotting Basics                      ########
######################################################################

### Plots in Matplotlib reside within a figure object, use plt.figure to create a new figure
fig = plt.figure()

### Create on, or more subplots using add_subplot, since you cannot create a blank figure
ax = fig.add_subplot(1, 1, 1)

### Call your subplot created above, with a specified graph type (hist(), scatter(), etc.) and then idenitfy the variable to use
ax.hist(full_data['spend'], bins = 5)

### Add labels and a title
plt.title('Spend Distribution')
plt.xlabel('Spend ($)')
plt.ylabel('#Commodity - Division Combinations')
plt.show()

In [ ]:
######## Plotting with Seaborn ########
### Seaborn plots do not require an explict statement to build the figure
### Read more about Seaborn plots in the link below
# https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python
sns.distplot(full_data['spend'])

In [ ]:
######## Scatter Plots ########

### Plots in Matplotlib reside within a figure object, use plt.figure to create a new figure
fig = plt.figure()

### Create on, or more subplots using add_subplot, since you cannot create a blank figure
ax = fig.add_subplot(1, 1, 1)

### Identify variables to use
### The first variable is the x-axis and the second is the y-axis
ax.scatter(full_data['spend'], full_data['visits'])

### Add labels and Title
plt.title('Spend vs Visits Distribution')
plt.xlabel('Spend ($)')
plt.ylabel('Visits (#)')
plt.show()
